# 第 7 章：双向与多层 RNN

> 增强 RNN 表达能力的两种重要技术

---

## 本章目标

完成本章后，你将能够：

- [ ] 理解双向 RNN 的原理和应用场景
- [ ] 掌握多层 RNN 的堆叠方式
- [ ] 实现 BiLSTM 和多层 GRU
- [ ] 在实际任务中选择合适的架构

---

## 为什么需要扩展 RNN？

**标准 RNN 的局限**：

1. **单向性**：只能利用过去的上下文，无法看到未来
2. **深度有限**：单层 RNN 的表达能力有限

**解决方案**：

| 扩展方式 | 解决问题 | 代价 |
|----------|----------|------|
| 双向 RNN | 利用双向上下文 | 2x 参数和计算量 |
| 多层 RNN | 增加模型深度 | N x 参数和计算量 |

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

np.random.seed(42)
torch.manual_seed(42)

print("环境准备完成！")

---

## 第一部分：双向 RNN (Bidirectional RNN)

### 1.1 双向 RNN 的动机

考虑填空任务：

```
"我 喜欢 吃 ___ 和 西瓜"
```

要预测空格处的词，我们需要：
- **前文**："我 喜欢 吃" → 后面可能是食物
- **后文**："和 西瓜" → 可能是另一种水果

单向 RNN 只能看到前文，双向 RNN 可以同时利用前后文！

In [ ]:
# 可视化：单向 vs 双向 RNN

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# 左图：单向 RNN
ax = axes[0]
ax.set_xlim(0, 12)
ax.set_ylim(0, 8)

positions = [2, 4, 6, 8, 10]
words = ['我', '喜欢', '吃', '___', '西瓜']

for i, (x, word) in enumerate(zip(positions, words)):
    # 隐状态节点
    circle = plt.Circle((x, 5), 0.5, fill=True, facecolor='lightblue', 
                         edgecolor='blue', linewidth=2)
    ax.add_patch(circle)
    ax.text(x, 5, f'$h_{i+1}$', ha='center', va='center', fontsize=10)
    
    # 输入
    ax.text(x, 2.5, word, ha='center', fontsize=12, 
            bbox=dict(boxstyle='round', facecolor='lightyellow' if word != '___' else 'lightcoral'))
    ax.annotate('', xy=(x, 4.5), xytext=(x, 3.2),
                arrowprops=dict(arrowstyle='->', color='green', lw=1.5))
    
    # 水平连接（只有向右）
    if i < len(positions) - 1:
        ax.annotate('', xy=(positions[i+1]-0.5, 5), xytext=(x+0.5, 5),
                    arrowprops=dict(arrowstyle='->', color='blue', lw=2))

ax.set_title('单向 RNN: 只能看到过去', fontsize=14, fontweight='bold')
ax.text(6, 7, '问题: 预测 "___" 时看不到 "西瓜"', ha='center', fontsize=11, color='red')
ax.axis('off')

# 右图：双向 RNN
ax = axes[1]
ax.set_xlim(0, 12)
ax.set_ylim(0, 8)

for i, (x, word) in enumerate(zip(positions, words)):
    # 前向隐状态（上层）
    circle = plt.Circle((x, 5.8), 0.4, fill=True, facecolor='lightblue', 
                         edgecolor='blue', linewidth=2)
    ax.add_patch(circle)
    ax.text(x, 5.8, f'$\\overrightarrow{{h}}_{i+1}$', ha='center', va='center', fontsize=8)
    
    # 后向隐状态（下层）
    circle = plt.Circle((x, 4.2), 0.4, fill=True, facecolor='lightcoral', 
                         edgecolor='red', linewidth=2)
    ax.add_patch(circle)
    ax.text(x, 4.2, f'$\\overleftarrow{{h}}_{i+1}$', ha='center', va='center', fontsize=8)
    
    # 输入
    ax.text(x, 2, word, ha='center', fontsize=12,
            bbox=dict(boxstyle='round', facecolor='lightyellow' if word != '___' else 'lightgreen'))
    ax.annotate('', xy=(x, 3.8), xytext=(x, 2.7),
                arrowprops=dict(arrowstyle='->', color='green', lw=1.5))
    
    # 前向连接（向右）
    if i < len(positions) - 1:
        ax.annotate('', xy=(positions[i+1]-0.4, 5.8), xytext=(x+0.4, 5.8),
                    arrowprops=dict(arrowstyle='->', color='blue', lw=2))
    
    # 后向连接（向左）
    if i > 0:
        ax.annotate('', xy=(positions[i-1]+0.4, 4.2), xytext=(x-0.4, 4.2),
                    arrowprops=dict(arrowstyle='->', color='red', lw=2))

ax.set_title('双向 RNN: 同时看过去和未来', fontsize=14, fontweight='bold')
ax.text(6, 7.2, '优势: 预测 "___" 时可以利用 "吃" 和 "西瓜"', ha='center', fontsize=11, color='green')
ax.text(6, 0.8, '$h_t = [\\overrightarrow{h}_t ; \\overleftarrow{h}_t]$ (拼接)', 
        ha='center', fontsize=11)
ax.axis('off')

plt.tight_layout()
plt.show()

### 1.2 双向 RNN 的数学表示

**前向 RNN**（从左到右）：
$$\overrightarrow{h}_t = \text{RNN}_{\rightarrow}(x_t, \overrightarrow{h}_{t-1})$$

**后向 RNN**（从右到左）：
$$\overleftarrow{h}_t = \text{RNN}_{\leftarrow}(x_t, \overleftarrow{h}_{t+1})$$

**合并输出**：
$$h_t = [\overrightarrow{h}_t ; \overleftarrow{h}_t] \quad \text{(拼接)}$$

或者：
$$h_t = \overrightarrow{h}_t + \overleftarrow{h}_t \quad \text{(相加)}$$

In [ ]:
# NumPy 实现双向 RNN

class BidirectionalRNN:
    """
    双向 RNN 的 NumPy 实现
    
    包含两个独立的 RNN：
    - 前向 RNN: 从 t=1 到 t=T
    - 后向 RNN: 从 t=T 到 t=1
    """
    
    def __init__(self, input_size, hidden_size):
        self.hidden_size = hidden_size
        
        # 前向 RNN 参数
        scale = np.sqrt(2.0 / (input_size + hidden_size))
        self.W_xh_forward = np.random.randn(hidden_size, input_size) * scale
        self.W_hh_forward = np.random.randn(hidden_size, hidden_size) * scale
        self.b_h_forward = np.zeros(hidden_size)
        
        # 后向 RNN 参数（独立的参数）
        self.W_xh_backward = np.random.randn(hidden_size, input_size) * scale
        self.W_hh_backward = np.random.randn(hidden_size, hidden_size) * scale
        self.b_h_backward = np.zeros(hidden_size)
    
    def forward(self, X):
        """
        双向前向传播
        
        参数:
            X: 输入序列, shape: (seq_len, input_size)
        
        返回:
            h_concat: 拼接后的隐状态, shape: (seq_len, 2 * hidden_size)
        """
        seq_len = X.shape[0]
        
        # ========================================
        # 前向传播（从左到右）
        # ========================================
        h_forward = np.zeros((seq_len, self.hidden_size))
        h_t = np.zeros(self.hidden_size)
        
        for t in range(seq_len):
            h_t = np.tanh(self.W_xh_forward @ X[t] + 
                          self.W_hh_forward @ h_t + 
                          self.b_h_forward)
            h_forward[t] = h_t
        
        # ========================================
        # 后向传播（从右到左）
        # ========================================
        h_backward = np.zeros((seq_len, self.hidden_size))
        h_t = np.zeros(self.hidden_size)
        
        for t in range(seq_len - 1, -1, -1):  # 从后往前
            h_t = np.tanh(self.W_xh_backward @ X[t] + 
                          self.W_hh_backward @ h_t + 
                          self.b_h_backward)
            h_backward[t] = h_t
        
        # ========================================
        # 拼接前向和后向的隐状态
        # ========================================
        h_concat = np.concatenate([h_forward, h_backward], axis=1)
        
        return h_concat, h_forward, h_backward


# 测试
print("=" * 60)
print("双向 RNN 测试")
print("=" * 60)

input_size = 4
hidden_size = 8
seq_len = 10

birnn = BidirectionalRNN(input_size, hidden_size)
X = np.random.randn(seq_len, input_size)

h_concat, h_forward, h_backward = birnn.forward(X)

print(f"\n输入形状: {X.shape}")
print(f"前向隐状态形状: {h_forward.shape}")
print(f"后向隐状态形状: {h_backward.shape}")
print(f"拼接后形状: {h_concat.shape}  # hidden_size * 2")

### 1.3 PyTorch 双向 RNN

In [ ]:
def pytorch_bidirectional_demo():
    """
    PyTorch 双向 RNN 使用演示
    """
    print("=" * 60)
    print("PyTorch 双向 RNN")
    print("=" * 60)
    
    batch_size = 4
    seq_len = 10
    input_size = 8
    hidden_size = 16
    
    # 单向 LSTM
    lstm_uni = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=False)
    
    # 双向 LSTM
    lstm_bi = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True)
    
    # 输入
    x = torch.randn(batch_size, seq_len, input_size)
    
    # 前向传播
    out_uni, (h_uni, c_uni) = lstm_uni(x)
    out_bi, (h_bi, c_bi) = lstm_bi(x)
    
    print("\n单向 LSTM:")
    print(f"  输出 shape: {out_uni.shape}  # (batch, seq, hidden)")
    print(f"  h_n shape: {h_uni.shape}     # (1, batch, hidden)")
    
    print("\n双向 LSTM:")
    print(f"  输出 shape: {out_bi.shape}  # (batch, seq, hidden*2)")
    print(f"  h_n shape: {h_bi.shape}     # (2, batch, hidden) - 前向和后向")
    
    # 分离前向和后向输出
    print("\n分离前向和后向:")
    forward_out = out_bi[:, :, :hidden_size]
    backward_out = out_bi[:, :, hidden_size:]
    print(f"  前向输出: {forward_out.shape}")
    print(f"  后向输出: {backward_out.shape}")
    
    # 参数量对比
    params_uni = sum(p.numel() for p in lstm_uni.parameters())
    params_bi = sum(p.numel() for p in lstm_bi.parameters())
    
    print(f"\n参数量:")
    print(f"  单向: {params_uni:,}")
    print(f"  双向: {params_bi:,}")
    print(f"  比值: {params_bi/params_uni:.1f}x")


pytorch_bidirectional_demo()

### 1.4 双向 RNN 的应用场景

In [ ]:
def bidirectional_applications():
    """
    双向 RNN 的应用场景
    """
    print("=" * 60)
    print("双向 RNN 适用场景")
    print("=" * 60)
    
    applications = """
    
┌─────────────────────────────────────────────────────────────┐
│                    ✓ 适合双向 RNN                          │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  1. 序列标注 (Sequence Labeling)                            │
│     - 词性标注 (POS Tagging)                                │
│     - 命名实体识别 (NER)                                    │
│     - 语义角色标注                                          │
│     原因: 每个位置的标签依赖前后文                          │
│                                                             │
│  2. 文本分类 / 情感分析                                     │
│     - 整个句子的分类                                        │
│     原因: 可以获得整个序列的完整表示                        │
│                                                             │
│  3. 机器阅读理解                                            │
│     - 问答系统中的上下文编码                                │
│     原因: 答案可能依赖问题的任何部分                        │
│                                                             │
│  4. 语音识别的编码器                                        │
│     - 处理完整的音频后再解码                                │
│     原因: 可以利用整个音频的上下文                          │
│                                                             │
└─────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────┐
│                    ✗ 不适合双向 RNN                         │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  1. 语言模型 / 文本生成                                     │
│     - 预测下一个词                                          │
│     原因: 生成时看不到未来的词                              │
│                                                             │
│  2. 实时流处理                                              │
│     - 实时语音识别                                          │
│     - 在线翻译                                              │
│     原因: 必须等待整个序列完成才能处理                      │
│                                                             │
│  3. 自回归解码器                                            │
│     - Seq2Seq 的解码器部分                                  │
│     原因: 解码是逐步进行的                                  │
│                                                             │
└─────────────────────────────────────────────────────────────┘
"""
    
    print(applications)


bidirectional_applications()

---

## 第二部分：多层 RNN (Stacked RNN)

### 2.1 多层 RNN 的动机

就像 CNN 中堆叠多层卷积可以学习更复杂的特征，堆叠多层 RNN 可以学习更抽象的序列表示。

In [ ]:
# 可视化：单层 vs 多层 RNN

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 左图：单层 RNN
ax = axes[0]
ax.set_xlim(0, 14)
ax.set_ylim(0, 10)

positions = [2, 5, 8, 11]

for i, x in enumerate(positions):
    # 单层节点
    circle = plt.Circle((x, 5), 0.6, fill=True, facecolor='lightblue', 
                         edgecolor='blue', linewidth=2)
    ax.add_patch(circle)
    ax.text(x, 5, f'$h_{{1,{i+1}}}$', ha='center', va='center', fontsize=10)
    
    # 输入
    ax.text(x, 2, f'$x_{i+1}$', ha='center', fontsize=12)
    ax.annotate('', xy=(x, 4.4), xytext=(x, 2.6),
                arrowprops=dict(arrowstyle='->', color='green', lw=1.5))
    
    # 输出
    ax.annotate('', xy=(x, 7.5), xytext=(x, 5.6),
                arrowprops=dict(arrowstyle='->', color='red', lw=1.5))
    ax.text(x, 8, f'$y_{i+1}$', ha='center', fontsize=12)
    
    # 水平连接
    if i < len(positions) - 1:
        ax.annotate('', xy=(positions[i+1]-0.6, 5), xytext=(x+0.6, 5),
                    arrowprops=dict(arrowstyle='->', color='blue', lw=2))

ax.set_title('单层 RNN', fontsize=14, fontweight='bold')
ax.text(7, 9.2, '一层隐藏层', ha='center', fontsize=12)
ax.axis('off')

# 右图：多层 RNN
ax = axes[1]
ax.set_xlim(0, 14)
ax.set_ylim(0, 10)

layers = 3
layer_colors = ['lightblue', 'lightgreen', 'lightyellow']
layer_y = [3, 5, 7]

for layer, (y, color) in enumerate(zip(layer_y, layer_colors)):
    for i, x in enumerate(positions):
        # 节点
        circle = plt.Circle((x, y), 0.5, fill=True, facecolor=color, 
                             edgecolor='black', linewidth=1.5)
        ax.add_patch(circle)
        ax.text(x, y, f'$h_{{L{layer+1}}}$', ha='center', va='center', fontsize=8)
        
        # 水平连接（同层）
        if i < len(positions) - 1:
            ax.annotate('', xy=(positions[i+1]-0.5, y), xytext=(x+0.5, y),
                        arrowprops=dict(arrowstyle='->', color='blue', lw=1.5))
        
        # 垂直连接（层与层之间）
        if layer < layers - 1:
            ax.annotate('', xy=(x, layer_y[layer+1]-0.5), xytext=(x, y+0.5),
                        arrowprops=dict(arrowstyle='->', color='gray', lw=1))

# 输入
for i, x in enumerate(positions):
    ax.text(x, 1, f'$x_{i+1}$', ha='center', fontsize=10)
    ax.annotate('', xy=(x, 2.5), xytext=(x, 1.5),
                arrowprops=dict(arrowstyle='->', color='green', lw=1.5))

# 输出
for i, x in enumerate(positions):
    ax.text(x, 9, f'$y_{i+1}$', ha='center', fontsize=10)
    ax.annotate('', xy=(x, 8.5), xytext=(x, 7.5),
                arrowprops=dict(arrowstyle='->', color='red', lw=1.5))

ax.set_title('多层 RNN (3 层)', fontsize=14, fontweight='bold')
ax.text(7, 9.5, '每层学习不同层次的表示', ha='center', fontsize=12)

# 层标签
for layer, y in enumerate(layer_y):
    ax.text(0.5, y, f'Layer {layer+1}', fontsize=10, va='center')

ax.axis('off')

plt.tight_layout()
plt.show()

### 2.2 多层 RNN 的数学表示

对于 $L$ 层的 RNN：

**第 1 层**：
$$h_t^{(1)} = \text{RNN}^{(1)}(x_t, h_{t-1}^{(1)})$$

**第 $l$ 层**（$l > 1$）：
$$h_t^{(l)} = \text{RNN}^{(l)}(h_t^{(l-1)}, h_{t-1}^{(l)})$$

即：上一层的输出作为下一层的输入。

**最终输出**：
$$y_t = \text{Linear}(h_t^{(L)})$$

In [ ]:
def multilayer_rnn_demo():
    """
    PyTorch 多层 RNN 演示
    """
    print("=" * 60)
    print("多层 RNN 演示")
    print("=" * 60)
    
    batch_size = 4
    seq_len = 10
    input_size = 8
    hidden_size = 16
    
    # 不同层数的 LSTM
    lstm_1 = nn.LSTM(input_size, hidden_size, num_layers=1, batch_first=True)
    lstm_2 = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True)
    lstm_4 = nn.LSTM(input_size, hidden_size, num_layers=4, batch_first=True)
    
    x = torch.randn(batch_size, seq_len, input_size)
    
    # 前向传播
    out_1, (h_1, c_1) = lstm_1(x)
    out_2, (h_2, c_2) = lstm_2(x)
    out_4, (h_4, c_4) = lstm_4(x)
    
    print("\n输出形状 (num_layers 不影响输出维度):")
    print(f"  1 层: output={out_1.shape}, h_n={h_1.shape}")
    print(f"  2 层: output={out_2.shape}, h_n={h_2.shape}")
    print(f"  4 层: output={out_4.shape}, h_n={h_4.shape}")
    
    # 参数量对比
    params = []
    for lstm, name in [(lstm_1, '1层'), (lstm_2, '2层'), (lstm_4, '4层')]:
        p = sum(p.numel() for p in lstm.parameters())
        params.append((name, p))
        print(f"\n{name} LSTM 参数量: {p:,}")
    
    # 访问每层的隐状态
    print("\n多层 LSTM 的隐状态结构:")
    print(f"  h_n 形状: (num_layers, batch, hidden)")
    print(f"  h_2[0]: 第 1 层的最终隐状态")
    print(f"  h_2[1]: 第 2 层的最终隐状态")
    
    # 可视化参数量
    fig, ax = plt.subplots(figsize=(8, 5))
    
    names = [p[0] for p in params]
    values = [p[1]/1000 for p in params]
    
    bars = ax.bar(names, values, color=['steelblue', 'coral', 'green'], edgecolor='black')
    
    ax.set_xlabel('层数', fontsize=12)
    ax.set_ylabel('参数量 (千)', fontsize=12)
    ax.set_title('多层 LSTM 参数量', fontsize=14)
    ax.grid(True, alpha=0.3, axis='y')
    
    for bar, val in zip(bars, values):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
                f'{val:.0f}k', ha='center', fontsize=11)
    
    plt.tight_layout()
    plt.show()


multilayer_rnn_demo()

### 2.3 Dropout 在多层 RNN 中的应用

In [ ]:
def dropout_in_rnn():
    """
    RNN 中的 Dropout
    """
    print("=" * 60)
    print("RNN 中的 Dropout")
    print("=" * 60)
    
    explanation = """
    PyTorch RNN 的 dropout 参数:
    
    - 应用在层与层之间（不是时间步之间）
    - 只有多层 RNN 时才有效（num_layers > 1）
    - 最后一层没有 dropout
    
    示例: 3 层 LSTM, dropout=0.2
    
    Layer 3:  h3 ────────────────────▶ output (无 dropout)
              ▲
              │ dropout (0.2)
    Layer 2:  h2 ────────────────────▶
              ▲
              │ dropout (0.2)
    Layer 1:  h1 ────────────────────▶
              ▲
              │ (无 dropout)
    Input:    x
    """
    
    print(explanation)
    
    # 代码示例
    print("\nPyTorch 代码:")
    print("-" * 40)
    
    # 3 层 LSTM with dropout
    lstm = nn.LSTM(input_size=32, hidden_size=64, 
                   num_layers=3, 
                   dropout=0.2,  # 层间 dropout
                   batch_first=True)
    
    print("lstm = nn.LSTM(input_size=32, hidden_size=64,")
    print("               num_layers=3, dropout=0.2)")
    
    # 训练 vs 推理
    x = torch.randn(4, 10, 32)
    
    lstm.train()
    out_train, _ = lstm(x)
    
    lstm.eval()
    out_eval, _ = lstm(x)
    
    print(f"\n训练模式输出: {out_train.shape}")
    print(f"评估模式输出: {out_eval.shape}")
    print("\n注意: 评估时 dropout 自动关闭！")


dropout_in_rnn()

---

## 第三部分：组合架构

### 3.1 双向多层 RNN

In [ ]:
def bidirectional_multilayer():
    """
    双向多层 RNN
    """
    print("=" * 60)
    print("双向多层 RNN (BiLSTM)")
    print("=" * 60)
    
    batch_size = 4
    seq_len = 10
    input_size = 8
    hidden_size = 16
    num_layers = 2
    
    # 双向多层 LSTM
    bilstm = nn.LSTM(input_size, hidden_size, 
                     num_layers=num_layers,
                     bidirectional=True,
                     batch_first=True)
    
    x = torch.randn(batch_size, seq_len, input_size)
    output, (h_n, c_n) = bilstm(x)
    
    print(f"\n配置: {num_layers} 层, 双向")
    print(f"\n输出形状:")
    print(f"  output: {output.shape}  # (batch, seq, hidden*2)")
    print(f"  h_n: {h_n.shape}  # (num_layers*2, batch, hidden)")
    print(f"  c_n: {c_n.shape}  # (num_layers*2, batch, hidden)")
    
    # h_n 的结构
    print("\nh_n 结构解析:")
    print("  h_n[0]: Layer 1 前向的最终状态")
    print("  h_n[1]: Layer 1 后向的最终状态")
    print("  h_n[2]: Layer 2 前向的最终状态")
    print("  h_n[3]: Layer 2 后向的最终状态")
    
    # 获取最终表示的常用方法
    print("\n获取最终表示:")
    
    # 方法 1: 取最后一层的前向和后向拼接
    last_forward = h_n[-2]  # (batch, hidden)
    last_backward = h_n[-1]  # (batch, hidden)
    final_repr = torch.cat([last_forward, last_backward], dim=1)
    print(f"  方法1 (拼接最后一层): {final_repr.shape}")
    
    # 方法 2: 取输出序列的最后一个时间步
    last_output = output[:, -1, :]
    print(f"  方法2 (输出最后时刻): {last_output.shape}")
    
    # 方法 3: 对输出序列做平均池化
    avg_pool = output.mean(dim=1)
    print(f"  方法3 (平均池化): {avg_pool.shape}")


bidirectional_multilayer()

### 3.2 实战：BiLSTM 文本分类

In [ ]:
class BiLSTMClassifier(nn.Module):
    """
    双向 LSTM 文本分类器
    
    结构:
    Embedding → BiLSTM → Dropout → Linear → Softmax
    """
    
    def __init__(self, vocab_size, embed_dim, hidden_size, num_classes, 
                 num_layers=2, dropout=0.3):
        super().__init__()
        
        # 词嵌入层
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # 双向 LSTM
        self.lstm = nn.LSTM(
            input_size=embed_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
        # 分类层（输入是 hidden_size * 2 因为是双向）
        self.fc = nn.Linear(hidden_size * 2, num_classes)
    
    def forward(self, x):
        """
        前向传播
        
        参数:
            x: 词索引序列, shape: (batch, seq_len)
        
        返回:
            logits: 分类 logits, shape: (batch, num_classes)
        """
        # 词嵌入: (batch, seq_len) → (batch, seq_len, embed_dim)
        embedded = self.embedding(x)
        
        # BiLSTM
        # output: (batch, seq_len, hidden*2)
        # h_n: (num_layers*2, batch, hidden)
        output, (h_n, c_n) = self.lstm(embedded)
        
        # 取最后一层的前向和后向状态拼接
        # h_n[-2]: 最后一层前向, h_n[-1]: 最后一层后向
        final_hidden = torch.cat([h_n[-2], h_n[-1]], dim=1)  # (batch, hidden*2)
        
        # Dropout + 分类
        out = self.dropout(final_hidden)
        logits = self.fc(out)
        
        return logits


# 测试模型
print("=" * 60)
print("BiLSTM 文本分类器测试")
print("=" * 60)

# 模型配置
vocab_size = 10000
embed_dim = 128
hidden_size = 256
num_classes = 5
num_layers = 2

model = BiLSTMClassifier(vocab_size, embed_dim, hidden_size, num_classes, num_layers)

# 模拟输入
batch_size = 8
seq_len = 50
x = torch.randint(0, vocab_size, (batch_size, seq_len))

# 前向传播
logits = model(x)

print(f"\n输入形状: {x.shape}")
print(f"输出形状: {logits.shape}")
print(f"\n总参数量: {sum(p.numel() for p in model.parameters()):,}")

# 打印模型结构
print("\n模型结构:")
print(model)

---

## 第四部分：实践建议

### 4.1 架构选择指南

In [ ]:
def architecture_guide():
    """
    RNN 架构选择指南
    """
    print("=" * 70)
    print("RNN 架构选择指南")
    print("=" * 70)
    
    guide = """
    
┌────────────────────────────────────────────────────────────────────┐
│                         层数选择                                   │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│  • 1-2 层: 大多数任务的起点                                        │
│  • 2-4 层: 复杂任务（机器翻译、语音识别）                          │
│  • 4+ 层: 需要残差连接防止梯度消失                                 │
│                                                                    │
│  经验法则: 从 2 层开始，如果过拟合减少，欠拟合增加                 │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘

┌────────────────────────────────────────────────────────────────────┐
│                       单向 vs 双向                                 │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│  单向:                                                             │
│    ✓ 语言模型、文本生成                                            │
│    ✓ 实时/流式处理                                                 │
│    ✓ 自回归解码                                                    │
│                                                                    │
│  双向:                                                             │
│    ✓ 序列标注（NER、POS）                                          │
│    ✓ 文本分类                                                      │
│    ✓ 编码器（Seq2Seq 的 encoder）                                  │
│    ✓ 机器阅读理解                                                  │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘

┌────────────────────────────────────────────────────────────────────┐
│                       常见配置                                     │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│  文本分类:                                                         │
│    BiLSTM, 2 层, hidden=256, dropout=0.3                           │
│                                                                    │
│  序列标注 (NER):                                                   │
│    BiLSTM-CRF, 2 层, hidden=256                                    │
│                                                                    │
│  机器翻译编码器:                                                   │
│    BiLSTM, 2-4 层, hidden=512                                      │
│                                                                    │
│  语言模型:                                                         │
│    单向 LSTM, 2-3 层, hidden=512-1024                              │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
"""
    
    print(guide)


architecture_guide()

---

## 本章总结

### 核心要点

1. **双向 RNN**：
   - 同时利用过去和未来的上下文
   - 输出维度翻倍：`hidden_size * 2`
   - 不适用于需要实时预测的任务

2. **多层 RNN**：
   - 增加模型深度，学习更抽象的表示
   - 层间可以添加 dropout
   - 通常 2-4 层就足够

3. **组合使用**：
   - 双向 + 多层 = BiLSTM/BiGRU
   - `h_n` 形状为 `(num_layers * num_directions, batch, hidden)`

### 代码模板

```python
# 双向多层 LSTM
lstm = nn.LSTM(
    input_size=embed_dim,
    hidden_size=hidden_size,
    num_layers=2,
    bidirectional=True,
    dropout=0.3,
    batch_first=True
)

output, (h_n, c_n) = lstm(x)
# output: (batch, seq, hidden*2)
# h_n: (num_layers*2, batch, hidden)

# 获取最终表示
final = torch.cat([h_n[-2], h_n[-1]], dim=1)  # (batch, hidden*2)
```

---

## 下一章预告

**第 8 章：语言模型**

用 RNN 构建语言模型：
- 预测下一个词
- 计算困惑度 (Perplexity)
- 文本生成

👉 [08_language_model.ipynb](./08_language_model.ipynb)